# AILib: Comprehensive Feature Showcase

This notebook demonstrates all the features of AILib, a simple and intuitive Python SDK for building LLM-powered applications.

## Table of Contents

1. [Setup and Installation](#setup)
2. [Basic LLM Completions](#basic-completions)
3. [Prompt Templates](#prompt-templates)
4. [Building Conversations with Prompt Builder](#prompt-builder)
5. [Session Management](#session-management)
6. [Chains: Sequential Operations](#chains)
7. [Tools and Decorators](#tools)
8. [Agents: Autonomous Problem Solving](#agents)
9. [Advanced Features](#advanced)
10. [Real-World Examples](#examples)


## 1. Setup and Installation <a id='setup'></a>

First, let's set up the environment and import the necessary modules.


In [1]:
# Install AILib (if not already installed)
# !pip install -e ..

# Set up environment variable for OpenAI API key
import os
from getpass import getpass

if not os.getenv("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API key: ")

In [2]:
# Import all AILib modules
from ailib import (
    # Core components
    OpenAIClient,
    Message,
    Role,
    Prompt,
    PromptTemplate,
    Session,
    # Chains
    Chain,
    # Agents and tools
    Agent,
    Tool,
    ToolRegistry,
    tool,
)

# Import helpers
from ailib.core.prompt import create_react_prompt, create_few_shot_prompt

print("AILib imported successfully!")

AILib imported successfully!


## 2. Basic LLM Completions <a id='basic-completions'></a>

Let's start with the simplest use case: getting a completion from an LLM.


In [4]:
# Initialize the OpenAI client
client = OpenAIClient(model="gpt-3.5-turbo")

# Create a simple message
messages = [
    Message(role=Role.SYSTEM, content="You are a helpful assistant."),
    Message(role=Role.USER, content="What is the capital of France?"),
]

# Get completion
response = client.complete(messages)

print(f"Response: {response.content}")
print(f"\nModel used: {response.model}")
print(f"Tokens used: {response.usage}")

Response: The capital of France is Paris.

Model used: gpt-3.5-turbo-0125
Tokens used: {'prompt_tokens': 24, 'completion_tokens': 7, 'total_tokens': 31}


### Streaming Responses

AILib supports streaming for real-time output:


In [4]:
# Stream a response token by token
messages = [Message(role=Role.USER, content="Write a haiku about programming")]

print("Streaming response:")
for token in client.stream(messages):
    print(token, end="", flush=True)
print("\n\nStreaming complete!")

Streaming response:
Infinite loops spin
Code dances across the screen
Solving problems, one

Streaming complete!


## 3. Prompt Templates <a id='prompt-templates'></a>

Prompt templates allow you to create reusable prompts with variable substitution.


In [5]:
# Create a simple template
template = PromptTemplate(
    "Translate the following {source_lang} text to {target_lang}: '{text}'"
)

# Show template variables
print(f"Template variables: {template.variables}")

# Format the template
formatted = template.format(
    source_lang="English", target_lang="French", text="Hello, world!"
)
print(f"\nFormatted prompt: {formatted}")

Template variables: ['target_lang', 'source_lang', 'text']

Formatted prompt: Translate the following English text to French: 'Hello, world!'


### Partial Templates

You can create partial templates by pre-filling some variables:


In [6]:
# Create a partial template with target language pre-filled
spanish_translator = template.partial(target_lang="Spanish")

print(f"Remaining variables: {spanish_translator.variables}")

# Use the partial template
translations = [
    spanish_translator.format(source_lang="English", text="Good morning"),
    spanish_translator.format(source_lang="French", text="Bonjour"),
]

for prompt in translations:
    print(prompt)

Remaining variables: ['source_lang', 'text']
Translate the following English text to Spanish: 'Good morning'
Translate the following French text to Spanish: 'Bonjour'


### Using Templates with LLM


In [7]:
# Create a message from template and get completion
message = spanish_translator.create_message(source_lang="English", text="How are you?")

response = client.complete([message])
print(f"Translation: {response.content}")

Translation: ¿Cómo estás?


## 4. Building Conversations with Prompt Builder <a id='prompt-builder'></a>

The Prompt builder provides a fluent API for constructing multi-message conversations.


In [8]:
# Build a conversation using the fluent API
prompt = Prompt()
prompt.add_system("You are a Python programming tutor.")
prompt.add_user("What is a list comprehension?")
prompt.add_assistant(
    "A list comprehension is a concise way to create lists in Python..."
)
prompt.add_user("Can you show me an example?")

# Get the messages
messages = prompt.build()

# Display the conversation
for i, msg in enumerate(messages):
    print(f"{i+1}. [{msg.role.value}]: {msg.content[:50]}...")

1. [system]: You are a Python programming tutor....
2. [user]: What is a list comprehension?...
3. [assistant]: A list comprehension is a concise way to create li...
4. [user]: Can you show me an example?...


### Using Templates in Conversations


In [9]:
# Build a conversation with templates
code_review_prompt = Prompt()
code_review_prompt.add_system("You are a code reviewer specializing in {language}.")
code_review_prompt.add_template(
    "Review this {language} code for best practices:\n```{language}\n{code}\n```",
    role=Role.USER,
    language="Python",
    code="def add(a,b): return a+b",
)

# Get completion
messages = code_review_prompt.build()
response = client.complete(messages)
print("Code Review:")
print(response.content)

Code Review:
This code looks concise and functional, but it could be improved for better readability and maintainability. Here are some suggestions:

1. **Whitespace**: Add whitespace around the `:` and `,` for better readability:
   ```python
   def add(a, b): return a + b
   ```

2. **Documentation**: Add a docstring to describe the function's purpose and parameters:
   ```python
   def add(a, b):
       """Add two numbers together."""
       return a + b
   ```

3. **Error Handling**: Consider adding error handling for cases where `a` or `b` are not numbers.

4. **Type Hints**: Consider adding type hints for the function parameters and return value for better code readability and maintainability:
   ```python
   def add(a: int, b: int) -> int:
       """Add two numbers together."""
       return a + b
   ```

5. **Pep8**: Ensure the code follows the PEP8 style guide for Python to maintain consistency.

Overall, the code is functional but could benefit from some enhancements for bett

## 5. Session Management <a id='session-management'></a>

Sessions help maintain conversation state and memory across interactions.


In [10]:
# Create a new session
session = Session()

print(f"Session ID: {session.session_id}")
print(f"Created at: {session.created_at}")

Session ID: ac3e282d-ad52-4d45-8345-31bf9eb0636a
Created at: 2025-07-29 07:50:58.632250


### Managing Conversation History


In [11]:
# Add messages to session
session.add_system_message("You are a helpful math tutor.")
session.add_user_message("What is the Pythagorean theorem?")

# Get response from LLM
response = client.complete(session.get_messages())
session.add_assistant_message(response.content)

print("First interaction:")
print(response.content[:200] + "...")

# Continue the conversation
session.add_user_message("Can you give me an example with numbers?")
response = client.complete(session.get_messages())
session.add_assistant_message(response.content)

print("\nSecond interaction (with context):")
print(response.content[:200] + "...")

print(f"\nTotal messages in session: {len(session)}")

First interaction:
The Pythagorean theorem states that in a right-angled triangle, the square of the length of the hypotenuse (the side opposite the right angle) is equal to the sum of the squares of the lengths of the ...

Second interaction (with context):
Sure! Let's consider a right-angled triangle with side lengths of 3 units, 4 units, and an unknown length for the hypotenuse (denoted as \( c \)).

Using the Pythagorean theorem, we have:

\( 3^2 + 4^...

Total messages in session: 5


### Session Memory Storage


In [12]:
# Store information in session memory
session.set_memory("user_name", "Alice")
session.set_memory("topic", "Pythagorean theorem")
session.set_memory("skill_level", "beginner")

# Retrieve memory
print(f"User: {session.get_memory('user_name')}")
print(f"Topic: {session.get_memory('topic')}")
print(f"Level: {session.get_memory('skill_level')}")

# Update multiple values
session.update_memory(
    {"examples_given": 2, "last_question": "Pythagorean theorem example"}
)

print(f"\nAll memory: {session._memory}")

User: Alice
Topic: Pythagorean theorem
Level: beginner

All memory: {'user_name': 'Alice', 'topic': 'Pythagorean theorem', 'skill_level': 'beginner', 'examples_given': 2, 'last_question': 'Pythagorean theorem example'}


### Session Persistence


In [13]:
# Convert session to dict (for saving)
session_data = session.to_dict()
print(f"Session data keys: {list(session_data.keys())}")

# Restore session from dict
restored_session = Session.from_dict(session_data)
print(f"\nRestored session ID: {restored_session.session_id}")
print(f"Restored messages: {len(restored_session)}")
print(f"Restored memory: {restored_session.get_memory('user_name')}")

Session data keys: ['session_id', 'created_at', 'messages', 'memory', 'metadata', 'trace']

Restored session ID: ac3e282d-ad52-4d45-8345-31bf9eb0636a
Restored messages: 5
Restored memory: Alice


## 6. Chains: Sequential Operations <a id='chains'></a>

Chains allow you to execute multiple prompts sequentially, with each step building on the previous.


In [14]:
# Create a simple chain
chain = Chain(client)
chain.add_system("You are a helpful assistant.")
chain.add_user("What is the capital of France?")

# Run the chain
result = chain.run()
print(f"Result: {result}")

Result: The capital of France is Paris.


### Multi-Step Chains with Context


In [15]:
# Create a multi-step chain where each step uses previous results
story_chain = (
    Chain(client)
    .add_system("You are a creative storyteller.")
    .add_user(
        "Generate a random character name for a fantasy story", name="character_name"
    )
    .add_user("Create a one-sentence backstory for {character_name}", name="backstory")
    .add_user(
        "Write a short adventure scene featuring {character_name}. Their backstory: {backstory}",
        name="scene",
    )
)

# Enable verbose mode to see each step
story_chain.verbose(True)

# Run the chain
final_scene = story_chain.run()
print("\n=== Final Scene ===")
print(final_scene)


[step_1] Sending 1 messages...
Last message: You are a creative storyteller....
[step_1] Response: Once upon a time, in a faraway land called Lumina, there was a young girl named Elara who had a special gift. Elara had the ability to communicate with animals, understanding their thoughts and feelin...

[character_name] Sending 3 messages...
Last message: Generate a random character name for a fantasy story...
[character_name] Response: Thornwyn Emberheart...

[backstory] Sending 5 messages...
Last message: Create a one-sentence backstory for Thornwyn Emberheart...
[backstory] Response: Thornwyn Emberheart, a fierce warrior with a mysterious past, was born under the light of a blood-red moon and raised by a pack of ancient wolves in the shadow of the forbidden Blackwood Forest....

[scene] Sending 7 messages...
Last message: Write a short adventure scene featuring Thornwyn Emberheart. Their backstory: Thornwyn Emberheart, a...
[scene] Response: Thornwyn Emberheart stood at the edge of 

### Chains with Processing Functions


In [16]:
import json


# Define processor functions
def extract_json(text: str) -> dict:
    """Extract JSON from LLM response."""
    # Find JSON in the response
    import re

    json_match = re.search(r"\{.*\}", text, re.DOTALL)
    if json_match:
        return json.loads(json_match.group())
    return {}


def format_product(data: dict) -> str:
    """Format product data nicely."""
    return f"**{data.get('name', 'Unknown')}** - ${data.get('price', 0):.2f}\n{data.get('description', '')}"


# Create a chain with processors
product_chain = (
    Chain(client)
    .add_system("You are a product data generator.")
    .add_user(
        "Generate a random product as JSON with fields: name, price, description",
        processor=extract_json,
        name="product_data",
    )
    .add_user(
        "The product data is: {product_data}. Create a marketing tagline for it.",
        name="tagline",
    )
)

# Run the chain
result = product_chain.run()

# Access intermediate results
print("Product Data:")
print(format_product(product_chain._context["product_data"]))
print(f"\nTagline: {result}")

Product Data:
**Luminous Smart LED Light Bulb** - $29.99
Transform your home with the Luminous Smart LED Light Bulb. This energy-efficient bulb can be controlled remotely via a smartphone app, allowing you to adjust brightness, color, and scheduling. With a lifespan of 25,000 hours, this smart bulb is the perfect addition to any smart home setup.

Tagline: "Illuminate Your Space, Control Your Ambiance - Experience the Future with the Luminous Smart LED Light Bulb!"


## 7. Tools and Decorators <a id='tools'></a>

Tools extend the capabilities of agents by providing functions they can call.


In [6]:
# Alternative approach: Define tools as regular functions first, then register them
# This approach works better for re-running cells in notebooks


def get_weather_func(city: str, unit: str = "celsius") -> str:
    """Get the current weather for a city."""
    import random

    temp = random.randint(15, 30) if unit == "celsius" else random.randint(59, 86)
    conditions = random.choice(["sunny", "cloudy", "partly cloudy", "rainy"])
    return f"The weather in {city} is {conditions} with a temperature of {temp}°{unit[0].upper()}"


def search_func(query: str, max_results: int = 3) -> str:
    """Perform a web search."""
    results = [f"Result {i+1}: Information about {query}" for i in range(max_results)]
    return "\n".join(results)


def calculate_func(expression: str) -> float:
    """Evaluate a mathematical expression."""
    allowed = {"__builtins__": {}}
    allowed.update(
        {"abs": abs, "round": round, "min": min, "max": max, "sum": sum, "pow": pow}
    )
    try:
        return float(eval(expression, allowed))
    except Exception as e:
        return f"Error: {str(e)}"


# Create a new registry (this clears any existing tools)
notebook_registry = ToolRegistry()

# Create tools manually
weather_tool = Tool(
    name="get_weather", description="Get weather for a city", func=get_weather_func
)
search_tool = Tool(name="web_search", description="Search the web", func=search_func)
calc_tool = Tool(
    name="calculate", description="Calculate math expressions", func=calculate_func
)

# Register tools
notebook_registry.register(weather_tool)
notebook_registry.register(search_tool)
notebook_registry.register(calc_tool)

print(f"Tools registered: {notebook_registry.list_tools()}")

Tools registered: ['get_weather', 'web_search', 'calculate']


# IMPORTANT: If you get "already registered" errors when re-running cells with @tool decorator,

# restart the kernel (Kernel -> Restart) to reload the updated ailib code.

# Alternatively, use the manual registration approach shown below.

# Define tools using the @tool decorator with custom registry

# First, let's try to clear any existing tools to allow re-running

try: # If tools exist, unregister them first
if 'notebook_registry' in globals():
notebook_registry = ToolRegistry() # Create fresh registry
except:
pass

# Create a fresh registry

notebook_registry = ToolRegistry()

@tool(registry=notebook_registry)
def get_weather(city: str, unit: str = "celsius") -> str:
"""Get the current weather for a city.

    Args:
        city: Name of the city
        unit: Temperature unit (celsius or fahrenheit)
    """
    # Mock implementation
    import random

    temp = random.randint(15, 30) if unit == "celsius" else random.randint(59, 86)
    conditions = random.choice(["sunny", "cloudy", "partly cloudy", "rainy"])
    return f"The weather in {city} is {conditions} with a temperature of {temp}°{unit[0].upper()}"

@tool(name="web_search", description="Search the web for information", registry=notebook_registry)
def search(query: str, max_results: int = 3) -> str:
"""Perform a web search.""" # Mock implementation
results = [f"Result {i+1}: Information about {query}" for i in range(max_results)]
return "\n".join(results)

@tool(registry=notebook_registry)
def calculate(expression: str) -> float:
"""Evaluate a mathematical expression.""" # Safe evaluation
allowed = {"**builtins**": {}}
allowed.update(
{"abs": abs, "round": round, "min": min, "max": max, "sum": sum, "pow": pow}
)
try:
return float(eval(expression, allowed))
except Exception as e:
return f"Error: {str(e)}"

print("Tools defined successfully!")
print(f"Tools in registry: {notebook_registry.list_tools()}")


# Create a tool manually

def get_time(timezone: str = "UTC") -> str:
"""Get current time in specified timezone."""
from datetime import datetime
from zoneinfo import ZoneInfo

    try:
        # Use zoneinfo (Python 3.9+) instead of pytz
        tz = ZoneInfo(timezone)
        time = datetime.now(tz)
        return time.strftime("%Y-%m-%d %H:%M:%S %Z")
    except:
        # Fallback to UTC
        return datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S UTC")

# Create Tool instance

time_tool = Tool(
name="get_time", description="Get current time in any timezone", func=get_time
)

# Test the tool

print("Manual tool test:")
print(time_tool.execute(timezone="US/Pacific"))
print(time_tool.execute(timezone="UTC"))


In [8]:
# Create a tool manually
def get_time(timezone: str = "UTC") -> str:
    """Get current time in specified timezone."""
    from datetime import datetime

    try:
        # For Python 3.9+
        from zoneinfo import ZoneInfo

        tz = ZoneInfo(timezone)
        time = datetime.now(tz)
        return time.strftime("%Y-%m-%d %H:%M:%S %Z")
    except ImportError:
        # Fallback for older Python versions
        return datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S UTC")


# Create Tool instance
time_tool = Tool(
    name="get_time", description="Get current time in any timezone", func=get_time
)

# Test the tool
print("Manual tool test:")
print(time_tool.execute(timezone="US/Pacific"))
print(time_tool.execute(timezone="UTC"))

Manual tool test:
2025-07-29 01:14:03 PDT
2025-07-29 08:14:03 UTC


### Tool Registry Management


In [9]:
# Create a custom tool registry
my_registry = ToolRegistry()

# Register tools
my_registry.register(time_tool)


# Define and register a tool with custom registry
@tool(registry=my_registry)
def translate(text: str, target_language: str) -> str:
    """Translate text to target language."""
    # Mock implementation
    translations = {
        "spanish": {"hello": "hola", "goodbye": "adiós", "thank you": "gracias"},
        "french": {"hello": "bonjour", "goodbye": "au revoir", "thank you": "merci"},
        "german": {
            "hello": "hallo",
            "goodbye": "auf wiedersehen",
            "thank you": "danke",
        },
    }

    lang = target_language.lower()
    text_lower = text.lower()

    if lang in translations and text_lower in translations[lang]:
        return translations[lang][text_lower]
    return f"[Translation of '{text}' to {target_language}]"


# List tools in registry
print(f"Tools in registry: {my_registry.list_tools()}")

# Get OpenAI function definitions
functions = my_registry.to_openai_functions()
print(f"\nOpenAI function format:")
for func in functions:
    print(f"- {func['name']}: {func['description']}")

Tools in registry: ['get_time', 'translate']

OpenAI function format:
- get_time: Get current time in any timezone
- translate: Translate text to target language.


## 8. Agents: Autonomous Problem Solving <a id='agents'></a>

Agents can autonomously decide which tools to use to accomplish tasks.


In [12]:
# Create an agent with tools from our notebook registry
agent = Agent(
    llm=OpenAIClient(model="gpt-4"), tools=notebook_registry, max_steps=5, verbose=True
)

print(
    f"Agent created with {len(agent.tool_registry.list_tools())} tools: {agent.tool_registry.list_tools()}"
)

Agent created with 3 tools: ['get_weather', 'web_search', 'calculate']


### Simple Agent Task


In [13]:
# Run a simple task
result = agent.run("What's the weather in Paris?")
print(f"\nFinal Answer: {result}")


--- Step 1 ---
Assistant: Thought: The user wants to know the weather in Paris. I should use the get_weather tool to find this information.
Action: get_weather
Action Input: {"city": "Paris"}

Executing tool: get_weather
Input: {"city": "Paris"}
Observation: The weather in Paris is rainy with a temperature of 29°C

--- Step 2 ---
Assistant: Thought: With the information gained from the tool, I can now provide the user with the answer.
Action: Final Answer
Action Input: The weather in Paris is rainy with a temperature of 29°C.

Final Answer: The weather in Paris is rainy with a temperature of 29°C.


### Complex Multi-Tool Task


In [14]:
# Run a task requiring multiple tools
complex_task = """
I'm planning a trip to Tokyo. Can you:
1. Check the weather there
2. Calculate the cost if flights are $1200 and hotels are $150/night for 5 nights
3. Search for top tourist attractions
"""

result = agent.run(complex_task)
print(f"\nFinal Answer:\n{result}")


--- Step 1 ---
Assistant: Thought: The user has asked for three different tasks. Let's start with the first one, checking the weather in Tokyo.

Action: get_weather
Action Input: {"city": "Tokyo"}

Executing tool: get_weather
Input: {"city": "Tokyo"}
Observation: The weather in Tokyo is rainy with a temperature of 20°C

--- Step 2 ---
Assistant: Thought: Now that I have provided the weather information for Tokyo, I will proceed to the second task, which is calculating the cost of flights and hotels.

Action: calculate
Action Input: {"expression": "1200 + (150 * 5)"}

Executing tool: calculate
Input: {"expression": "1200 + (150 * 5)"}
Observation: 1950.0

--- Step 3 ---
Assistant: Thought: Now that I have calculated the cost of flights and hotels, I will proceed to the third task, which is searching for top tourist attractions in Tokyo.

Action: web_search
Action Input: {"query": "Top tourist attractions in Tokyo"}

Executing tool: web_search
Input: {"query": "Top tourist attractions i

# Create a specialized math tutor agent with its own registry

# Create a fresh registry each time to allow re-running

math_registry = ToolRegistry()

# Define functions first (without decorator)

def solve_equation_func(equation: str) -> str:
"""Solve algebraic equations step by step.""" # Mock implementation
steps = [
f"Given: {equation}",
"Step 1: Isolate the variable",
"Step 2: Simplify both sides",
"Step 3: Solve for x",
"Solution: x = 5",
]
return "\n".join(steps)

def explain_concept_func(concept: str) -> str:
"""Explain a mathematical concept."""
explanations = {
"derivative": "The derivative measures the rate of change of a function.",
"integral": "The integral represents the area under a curve.",
"limit": "A limit describes the value a function approaches as input approaches a point.",
}
return explanations.get(concept.lower(), f"Explanation of {concept}...")

def math_calculate_func(expression: str) -> float:
"""Evaluate a mathematical expression.""" # Safe evaluation
allowed = {"**builtins**": {}}
allowed.update(
{"abs": abs, "round": round, "min": min, "max": max, "sum": sum, "pow": pow}
)
try:
return float(eval(expression, allowed))
except Exception as e:
return f"Error: {str(e)}"

# Create and register tools manually

solve_tool = Tool(name="solve_equation", description="Solve algebraic equations", func=solve_equation_func)
explain_tool = Tool(name="explain_concept", description="Explain math concepts", func=explain_concept_func)
calc_tool = Tool(name="math_calculate", description="Calculate expressions", func=math_calculate_func)

math_registry.register(solve_tool)
math_registry.register(explain_tool)
math_registry.register(calc_tool)

# Create math tutor agent with verbose=True to see what's happening

math_tutor = Agent(llm=client, tools=math_registry, max_steps=6, verbose=True)

# Test the math tutor

question = "Can you explain what a derivative is and then solve the equation 2x + 5 = 15?"
print("Question:", question)
print("\n" + "="\*50 + "\n")

answer = math_tutor.run(question)
print(f"\nFinal Answer: {answer}")


In [3]:
# Create a specialized math tutor agent with its own registry# Create a fresh registry each time to allow re-runningmath_registry = ToolRegistry()# Define functions first (without decorator)def solve_equation_func(equation: str) -> str:    """Solve algebraic equations step by step."""    # Mock implementation    steps = [        f"Given: {equation}",        "Step 1: Isolate the variable",        "Step 2: Simplify both sides",        "Step 3: Solve for x",        "Solution: x = 5",    ]    return "\n".join(steps)def explain_concept_func(concept: str) -> str:    """Explain a mathematical concept."""    explanations = {        "derivative": "The derivative measures the rate of change of a function.",        "integral": "The integral represents the area under a curve.",        "limit": "A limit describes the value a function approaches as input approaches a point.",    }    return explanations.get(concept.lower(), f"Explanation of {concept}...")def math_calculate_func(expression: str) -> float:    """Evaluate a mathematical expression."""    # Safe evaluation    allowed = {"__builtins__": {}}    allowed.update(        {"abs": abs, "round": round, "min": min, "max": max, "sum": sum, "pow": pow}    )    try:        return float(eval(expression, allowed))    except Exception as e:        return f"Error: {str(e)}"# Create and register tools manuallysolve_tool = Tool(name="solve_equation", description="Solve algebraic equations", func=solve_equation_func)explain_tool = Tool(name="explain_concept", description="Explain math concepts", func=explain_concept_func)calc_tool = Tool(name="math_calculate", description="Calculate expressions", func=math_calculate_func)math_registry.register(solve_tool)math_registry.register(explain_tool)math_registry.register(calc_tool)# Create math tutor agent with verbose=True to see what's happeningmath_tutor = Agent(llm=client, tools=math_registry, max_steps=6, verbose=True)# Test the math tutorquestion = "Can you explain what a derivative is and then solve the equation 2x + 5 = 15?"print("Question:", question)print("\n" + "="*50 + "\n")answer = math_tutor.run(question)print(f"\nFinal Answer: {answer}")

NameError: name 'client' is not defined

## 9. Advanced Features <a id='advanced'></a>

Let's explore some advanced patterns and techniques.


### Async Operations


In [ ]:
import asyncio


async def async_example():
    """Demonstrate async operations."""
    # Async completion
    messages = [Message(role=Role.USER, content="Tell me a joke")]
    response = await client.acomplete(messages)
    print(f"Async response: {response.content}")

    # Async streaming
    print("\nAsync streaming:")
    async for token in client.astream(messages):
        print(token, end="", flush=True)
    print()

    # Async chain
    chain = Chain(client).add_user("What is 2+2?")
    result = await chain.arun()
    print(f"\nAsync chain result: {result}")


# Run async example
await async_example()

### Custom LLM Clients


In [ ]:
from ailib.core import LLMClient, CompletionResponse
from typing import List, Iterator, AsyncIterator, Optional, Any, Dict


class MockLLMClient(LLMClient):
    """A mock LLM client for testing."""

    def __init__(self, model: str = "mock-model"):
        super().__init__(model)
        self.responses = {
            "greeting": "Hello! How can I help you today?",
            "math": "The answer is 42.",
            "default": "I understand your question.",
        }

    def complete(self, messages: List[Message], **kwargs) -> CompletionResponse:
        # Simple keyword matching
        last_message = messages[-1].content.lower()

        if "hello" in last_message or "hi" in last_message:
            response = self.responses["greeting"]
        elif "math" in last_message or "calculate" in last_message:
            response = self.responses["math"]
        else:
            response = self.responses["default"]

        return CompletionResponse(
            content=response,
            model=self.model,
            usage={"prompt_tokens": 10, "completion_tokens": 5, "total_tokens": 15},
        )

    async def acomplete(self, messages: List[Message], **kwargs) -> CompletionResponse:
        return self.complete(messages, **kwargs)

    def stream(self, messages: List[Message], **kwargs) -> Iterator[str]:
        response = self.complete(messages, **kwargs)
        for char in response.content:
            yield char

    async def astream(self, messages: List[Message], **kwargs) -> AsyncIterator[str]:
        response = self.complete(messages, **kwargs)
        for char in response.content:
            yield char


# Test the mock client
mock_client = MockLLMClient()
response = mock_client.complete([Message(role=Role.USER, content="Hello there!")])
print(f"Mock response: {response.content}")

### Helper Functions


In [ ]:
# ReAct prompt helper
tools = ["search", "calculator", "weather"]
messages = create_react_prompt(
    question="What's the population of Paris multiplied by 2?", tools=tools
)

print("ReAct Prompt:")
for msg in messages:
    print(f"\n[{msg.role.value}]:\n{msg.content[:200]}...")

In [ ]:
# Few-shot learning helper
examples = [
    {"input": "happy", "output": "sad"},
    {"input": "big", "output": "small"},
    {"input": "hot", "output": "cold"},
]

messages = create_few_shot_prompt(
    instruction="Find the opposite word", examples=examples, query="fast"
)

print("Few-shot Prompt:")
for i, msg in enumerate(messages):
    print(f"{i+1}. [{msg.role.value}]: {msg.content}")

# Get completion
response = client.complete(messages)
print(f"\nAnswer: {response.content}")

## 10. Real-World Examples <a id='examples'></a>

Let's build some practical applications using AILib.


### Example 1: Code Documentation Generator


In [ ]:
def build_doc_generator():
    """Build a code documentation generator."""

    # Create a documentation template
    doc_template = PromptTemplate(
        """
Generate comprehensive documentation for this {language} code:

```{language}
{code}
```

Include:
1. Brief description
2. Parameters/Arguments
3. Return value
4. Example usage
"""
    )

    # Create a chain for generating docs
    doc_chain = (
        Chain(client)
        .add_system("You are a technical documentation expert.")
        .add_template(doc_template, language="Python", code="{code}")
    )

    return doc_chain


# Test the doc generator
doc_gen = build_doc_generator()

sample_code = """
def fibonacci(n: int) -> int:
    if n <= 1:
        return n
    return fibonacci(n-1) + fibonacci(n-2)
"""

documentation = doc_gen.run(code=sample_code)
print("Generated Documentation:")
print(documentation)

### Example 2: Interactive Tutoring System


In [ ]:
class TutoringSystem:
    """An interactive tutoring system using AILib."""

    def __init__(self, subject: str):
        self.subject = subject
        self.session = Session()
        self.client = OpenAIClient(model="gpt-3.5-turbo")

        # Initialize session
        self.session.add_system_message(
            f"""You are an expert {subject} tutor.
            Adapt your explanations to the student's level.
            Ask follow-up questions to ensure understanding."""
        )

        # Track progress
        self.session.set_memory("topics_covered", [])
        self.session.set_memory("student_level", "unknown")

    def ask_question(self, question: str) -> str:
        """Process a student question."""
        # Add question to session
        self.session.add_user_message(question)

        # Get response
        response = self.client.complete(self.session.get_messages())
        self.session.add_assistant_message(response.content)

        # Update topics covered
        topics = self.session.get_memory("topics_covered")
        if question.lower() not in str(topics).lower():
            topics.append(question[:50])  # Store first 50 chars
            self.session.set_memory("topics_covered", topics)

        return response.content

    def get_summary(self) -> str:
        """Get a summary of the tutoring session."""
        topics = self.session.get_memory("topics_covered")

        summary_prompt = f"""
        Based on our conversation, provide a brief summary of:
        1. Topics we covered: {topics}
        2. Key concepts explained
        3. Suggested next steps for the student
        """

        self.session.add_user_message(summary_prompt)
        response = self.client.complete(self.session.get_messages())

        return response.content


# Create a math tutor
tutor = TutoringSystem("Mathematics")

# Simulate a tutoring session
print("=== Math Tutoring Session ===")
print()

response1 = tutor.ask_question("What is calculus?")
print("Student: What is calculus?")
print(f"Tutor: {response1[:300]}...\n")

response2 = tutor.ask_question("Can you give me an example of a derivative?")
print("Student: Can you give me an example of a derivative?")
print(f"Tutor: {response2[:300]}...\n")

# Get session summary
print("=== Session Summary ===")
summary = tutor.get_summary()
print(summary)

### Example 3: Research Assistant Agent


In [ ]:
# Create a research assistant with custom tools and registry
research_registry = ToolRegistry()


@tool(registry=research_registry)
def analyze_topic(topic: str) -> str:
    """Analyze a research topic and identify key areas."""
    return f"""
    Analysis of '{topic}':
    - Main concepts: [Key concepts related to {topic}]
    - Research areas: [Current research directions]
    - Applications: [Practical applications]
    - Challenges: [Open problems and challenges]
    """


@tool(registry=research_registry)
def find_papers(topic: str, year_start: int = 2020) -> str:
    """Find relevant research papers on a topic."""
    # Mock implementation
    papers = [
        f"'{topic}: A Comprehensive Survey' (2023) - Smith et al.",
        f"'Advances in {topic}' (2022) - Johnson & Lee",
        f"'Future Directions for {topic}' (2024) - Brown et al.",
    ]
    return "Relevant papers:\n" + "\n".join(f"- {p}" for p in papers)


@tool(registry=research_registry)
def summarize_findings(papers: str, focus: str = "general") -> str:
    """Summarize research findings."""
    return f"""
    Summary of findings (focus: {focus}):
    1. Current state of the field shows significant progress
    2. Key innovations include new methodologies and applications
    3. Future work should address scalability and practical deployment
    """


# Add the search tool to research registry
@tool(name="research_search", registry=research_registry)
def research_search(query: str, max_results: int = 3) -> str:
    """Perform a web search for research."""
    # Mock implementation
    results = [f"Result {i+1}: Research about {query}" for i in range(max_results)]
    return "\n".join(results)


# Create research assistant
research_assistant = Agent(
    llm=OpenAIClient(model="gpt-4"), tools=research_registry, max_steps=6
)

# Conduct research
research_query = """
I need to research 'quantum computing applications in cryptography'.
Please analyze the topic, find relevant papers from 2022 onwards,
and summarize the key findings focusing on practical applications.
"""

print("=== Research Assistant ===")
research_result = research_assistant.run(research_query)
print(f"\nResearch Summary:\n{research_result}")

## Conclusion

This notebook has demonstrated the comprehensive features of AILib:

1. **Simple API**: Intuitive interfaces for all components
2. **Flexibility**: From basic completions to complex agents
3. **Extensibility**: Easy to add custom tools and clients
4. **Type Safety**: Full type hints throughout
5. **Production Ready**: Sessions, error handling, and async support

AILib provides a cleaner, more Pythonic alternative to existing frameworks while maintaining powerful capabilities for building LLM applications.

### Next Steps

- Explore the remaining low-priority features (validation, safety, tracing)
- Build your own custom tools and agents
- Integrate with your existing applications
- Contribute to the project on GitHub

Happy building with AILib! 🚀
